In [20]:
import pandas as pd
import plotly.graph_objects as go
import utils


In [21]:
plot_cols = ['ENTRY', 'STOPLOSS', 'TAKEPROFIT']
plot_colours = ['#043ef9', '#eb5334', '#34eb37']

def plot_candles(df_plot):
    fig = go.Figure()
    fig.add_trace(go.Candlestick(
        x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
        line=dict(width=1), opacity=1,
        increasing_fillcolor='#24A06B',
        decreasing_fillcolor="#CC2E3C",
        increasing_line_color='#2EC886',
        decreasing_line_color='#FF3A4C'
    ))
    for i in range(0, 3):
        fig.add_trace(go.Scatter(
            x=df_buys.time,
            y=df_buys[plot_cols[i]],
            mode='markers',
            marker=dict(color=plot_colours[i],size=12)
        ))
    for i in range(0, 3):
        fig.add_trace(go.Scatter(
            x=df_sells.time,
            y=df_sells[plot_cols[i]],
            mode='markers',
            marker=dict(color=plot_colours[i],size=12)
        ))
    fig.update_layout(width=1000,height=400,
        margin=dict(l=10,b=10,t=10),
        font=dict(size=10,color="#e1e1e1"),
        paper_bgcolor="#1e1e1e",
        plot_bgcolor="#1e1e1e")
    fig.update_xaxes(
        gridcolor="#1f292f",
        showgrid=True,fixedrange=True,rangeslider=dict(visible=False),
        rangebreaks=[
            dict(bounds=["sat", "mon"])
        ]
    )
    fig.update_yaxes(
        gridcolor="#1f292f",
        showgrid=True,
    )
    fig.show()

In [22]:
pair = "USD_JPY"
granularity = "H4"
df_raw = pd.read_pickle(utils.get_his_data_filename(pair, granularity))


In [23]:
non_cols = ['time', 'volume']
mod_cols = [x for x in df_raw.columns if x not in non_cols]
df_raw[mod_cols] = df_raw[mod_cols].apply(pd.to_numeric)

In [24]:
df_raw.columns

Index(['time', 'volume', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'bid_o', 'bid_h',
       'bid_l', 'bid_c', 'ask_o', 'ask_h', 'ask_l', 'ask_c'],
      dtype='object')

In [25]:
df = df_raw[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'bid_c', 'ask_c']].copy()

In [26]:
df.head()

,time,mid_o,mid_h,mid_l,mid_c,bid_c,ask_c
0,2020-01-01T22:00:00.000000000Z,108.651,108.761,108.607,108.673,108.667,108.679
1,2020-01-02T02:00:00.000000000Z,108.671,108.758,108.658,108.742,108.736,108.747
2,2020-01-02T06:00:00.000000000Z,108.744,108.842,108.696,108.839,108.833,108.845
3,2020-01-02T10:00:00.000000000Z,108.836,108.866,108.658,108.694,108.688,108.700
4,2020-01-02T14:00:00.000000000Z,108.697,108.748,108.212,108.529,108.523,108.535


In [27]:
SLOSS = 0.4
TPROFIT = 0.8
ENTRY_PRC = 0.1

def direction(row):
    if row.mid_c > row.mid_o:
        return 1
    return -1

def get_signal(row):
    if row.mid_h_prev > row.mid_h and row.mid_l_prev < row.mid_l:
        return row.DIRECTION_prev
    return 0

def get_entry_stop(row):
    if row.SIGNAL == 1:
        return (row.RANGE_prev * ENTRY_PRC) + row.mid_h_prev
    elif row.SIGNAL == -1:
        return row.mid_l_prev - (row.RANGE_prev * ENTRY_PRC)
    else:
        return 0
    
def get_stop_loss(row):
    if row.SIGNAL == 1:
        return row.ENTRY - (row.RANGE_prev * SLOSS)
    elif row.SIGNAL == -1:
        return row.ENTRY + (row.RANGE_prev * SLOSS)
    else:
        return 0
    
def get_take_profit(row):
    if row.SIGNAL == 1:
        return row.ENTRY + (row.RANGE_prev * TPROFIT)
    elif row.SIGNAL == -1:
        return row.ENTRY - (row.RANGE_prev * TPROFIT)
    else:
        return 0   

In [28]:
df = df_raw[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'bid_c', 'ask_c']].copy()
df['RANGE'] = df.mid_h - df.mid_l
df['mid_h_prev'] = df.mid_h.shift(1)
df['mid_l_prev'] = df.mid_l.shift(1)
df['RANGE_prev'] = df.RANGE.shift(1)
df['DIRECTION'] = df.apply(direction, axis=1)
df['DIRECTION_prev'] = df.DIRECTION.shift(1).fillna(0).astype(int)
df.dropna(inplace=True)
df['SIGNAL'] = df.apply(get_signal, axis=1)
df.reset_index(drop=True, inplace=True)


In [29]:
df['ENTRY'] = df.apply(get_entry_stop, axis=1)
df['STOPLOSS'] = df.apply(get_stop_loss, axis=1)
df['TAKEPROFIT'] = df.apply(get_take_profit, axis=1)

In [30]:
df[df.SIGNAL!=0].head()

,time,mid_o,mid_h,mid_l,mid_c,bid_c,ask_c,RANGE,mid_h_prev,mid_l_prev,RANGE_prev,DIRECTION,DIRECTION_prev,SIGNAL,ENTRY,STOPLOSS,TAKEPROFIT
0,2020-01-02T02:00:00.000000000Z,108.671,108.758,108.658,108.742,108.736,108.747,0.100,108.761,108.607,0.154,1,1,1,108.7764,108.7148,108.8996
4,2020-01-02T18:00:00.000000000Z,108.526,108.572,108.494,108.561,108.511,108.611,0.078,108.748,108.212,0.536,1,-1,-1,108.1584,108.3728,107.7296
12,2020-01-06T02:00:00.000000000Z,107.950,108.104,107.932,108.058,108.052,108.064,0.172,108.122,107.772,0.350,1,1,1,108.1570,108.0170,108.4370
14,2020-01-06T10:00:00.000000000Z,107.988,108.164,107.982,108.120,108.114,108.126,0.182,108.179,107.926,0.253,1,-1,-1,107.9007,108.0019,107.6983
20,2020-01-07T10:00:00.000000000Z,108.466,108.483,108.368,108.438,108.432,108.444,0.115,108.500,108.262,0.238,-1,1,1,108.5238,108.4286,108.7142


In [31]:
class Trade():
    def __init__(self, row):
        self.candle_date = row.time
        self.direction = row.SIGNAL
        self.entry = row.ENTRY
        self.TP = row.TAKEPROFIT
        self.SL = row.STOPLOSS
        self.running = False
        self.result = None
        self.stopped = None
        
    def update(self, row):
        if self.running == True:
            self.update_result(row)
        else:
            self.check_entry(row)
            
    def check_entry(self, row):
        if self.direction == 1 and row.mid_c >= self.entry or self.direction == -1 and row.mid_c <= self.entry:
            self.index = row.name
            self.opened = row.time
            self.running = True
            
    def update_result(self, row):
        if self.direction == 1:
            if row.mid_c >= self.TP:
                self.result = 2.0
            elif row.mid_c <= self.SL:
                self.result = -1.0
        else:
            if row.mid_c <= self.TP:
                self.result = 2.0
            elif row.mid_c >= self.SL:
                self.result = -1.0
        
        if self.result is not None:
            self.running = False
            self.stopped = row.time

In [32]:
open_trades = []
closed_trades = []

for index, row in df.iterrows():
    for ot in open_trades:
        ot.update(row)
        if ot.stopped is not None:
            closed_trades.append(ot)
    
    open_trades = [x for x in open_trades if x.stopped is None]
    
    if row.SIGNAL != 0:
        ope_trades = [x for x in open_trades if x.running == True]
        open_trades.append(Trade(row))


In [33]:
len(closed_trades)

747

In [34]:
df_trades = pd.DataFrame.from_dict([vars(x) for x in closed_trades])

In [35]:
df[df.SIGNAL!=0].to_pickle("USD_JPY_H4_trades.pkl")

In [36]:
df_plot = df.iloc[0:60]
df_buys = df_plot[df_plot.SIGNAL ==1]
df_sells = df_plot[df_plot.SIGNAL == -1]

In [37]:
df_trades.head()

,candle_date,direction,entry,TP,SL,running,result,stopped,index,opened
0,2020-01-02T02:00:00.000000000Z,1,108.7764,108.8996,108.7148,False,-1.0,2020-01-02T10:00:00.000000000Z,1,2020-01-02T06:00:00.000000000Z
1,2020-01-02T18:00:00.000000000Z,-1,108.1584,107.7296,108.3728,False,-1.0,2020-01-06T22:00:00.000000000Z,6,2020-01-03T02:00:00.000000000Z
2,2020-01-06T02:00:00.000000000Z,1,108.1570,108.4370,108.0170,False,2.0,2020-01-07T06:00:00.000000000Z,15,2020-01-06T14:00:00.000000000Z
3,2020-01-07T10:00:00.000000000Z,1,108.5238,108.7142,108.4286,False,-1.0,2020-01-07T22:00:00.000000000Z,21,2020-01-07T14:00:00.000000000Z
4,2020-01-07T18:00:00.000000000Z,1,108.6610,108.9090,108.5370,False,2.0,2020-01-08T14:00:00.000000000Z,26,2020-01-08T10:00:00.000000000Z


In [38]:
plot_candles(df_plot)